# PIFu: Pixel-Aligned Implicit Function for High-Resolution Clothed Human Digitization

**Original Repo**: https://github.com/shunsukesaito/PIFu

**Provided By**: https://github.com/BreezeWhite/PIFu

**Paper**: https://arxiv.org/pdf/1905.05172.pdf

In [ ]:
#@title Setup

!git clone https://github.com/BreezeWhite/PIFu

%cd PIFu
!sh ./scripts/download_trained_model.sh


# Install pytorch3d for visualization
import sys
import torch
pyt_version_str=torch.__version__.split("+")[0].replace(".", "")
version_str="".join([
    f"py3{sys.version_info.minor}_cu",
    torch.version.cuda.replace(".",""),
    f"_pyt{pyt_version_str}"
])
!pip install fvcore iopath
!pip install --no-index --no-cache-dir pytorch3d -f https://dl.fbaipublicfiles.com/pytorch3d/packaging/wheels/{version_str}/download.html

!pip install git+https://github.com/BreezeWhite/segpeo

In [ ]:
#@title Upload Image

import os
from google.colab import files
import IPython.display as dsp

ff = files.upload()
img_path = list(ff.keys())[0]
img_basename = os.path.basename(img_path)
img_name = os.path.splitext(img_basename)[0]

img_store_folder = os.path.join('images', img_name)
img_store_path = os.path.join(img_store_folder, img_basename)
os.makedirs(img_store_folder, exist_ok=True)
os.rename(img_path, img_store_path)

dsp.display(dsp.Image(img_store_path))

In [ ]:
#@title Run

# !segpeo --image-path {img_store_path}

%run apps/eval.py --test_folder_path {img_store_folder} --name {img_name}

from lib.colab_util import set_renderer, generate_video_from_obj, video

obj_path = f'./results/{img_name}/result_{img_name}.obj'
video_path = f'./results/{img_name}/result_{img_name}.mp4'
video_display_path = f'./results/{img_name}/result_{img_name}_display.mp4'

renderer = set_renderer()
generate_video_from_obj(obj_path, video_path, renderer)

# we cannot play a mp4 video generated by cv2
!ffmpeg -i $video_path -vcodec libx264 $video_display_path -y -loglevel quiet
video(video_display_path)